In [ ]:
#HERE IS PRESENTED THE WORKFLOW OF HANDLING TASK 2 Q1-Q2 FROM STAŻ BSH ASSIGNMENT

In [2]:
import pandas as pd
import numpy as np
import random

C:\Users\aksel\AppData\Local\Temp\ipykernel_3988\605883380.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
random.seed(42)

In [4]:
appliance_ids = [random.randint(1000000, 9999999) for _ in range(10)]
model_ids = [random.choice(["Bosch VF67", "Siemens GH41", "Whirlpool XY12"]) for _ in range(10)]
cookzone_ids = [random.choice(["A", "B", "C", "D", "N/A"]) for _ in range(10)]
message_texts = [random.choice(["Activated", "Deactivated", "Ustaw czas!", "Błąd, wyczyść płytę", f"Power set to {random.randint(1, 9)}"]) for _ in range(10)]



In [5]:
start = pd.Timestamp('2023-01-01')
end = pd.Timestamp('2023-12-31')
event_datetimes = [start + pd.Timedelta(days=random.randint(0, 365),
                                        hours=random.randint(0, 23),
                                        minutes=random.randint(0, 59),
                                        seconds=random.randint(0, 59),
                                        milliseconds=random.randint(0, 999),
                                        microseconds=random.randint(0, 999999)) for _ in range(10)]


In [6]:
appliance_id_series = pd.Series(appliance_ids, name='APPLIANCE_ID')
model_id_series = pd.Series(model_ids, name='MODEL_ID')
cookzone_id_series = pd.Series(cookzone_ids, name='COOKZONE_ID')
message_text_series = pd.Series(message_texts, name='MESSAGE_TEXT')
event_datetime_series = pd.Series(event_datetimes, name='EVENT_DATETIME')


In [15]:
task = pd.concat([appliance_id_series, model_id_series, cookzone_id_series, message_text_series, event_datetime_series], axis=1)


In [16]:
unique_values = {
    'APPLIANCE_ID': df['APPLIANCE_ID'].unique(),
    'MODEL_ID': df['MODEL_ID'].unique(),
    'COOKZONE_ID': df['COOKZONE_ID'].unique(),
    'MESSAGE_TEXT': df['MESSAGE_TEXT'].unique()
}

print("\nUnikalne wartości w poszczególnych kolumnach:")
for column, values in unique_values.items():
    print(f"{column}: {values}")



Unikalne wartości w poszczególnych kolumnach:
APPLIANCE_ID: [2867825 1419610 5614226 5108603 4744854 3341057 2719583 2458591 8078673
 1533224]
MODEL_ID: ['Bosch VF67' 'Whirlpool XY12']
COOKZONE_ID: ['N/A' 'D' 'B' 'C' 'A']
MESSAGE_TEXT: ['Deactivated' 'Ustaw czas!' 'Activated' 'Power set to 8'
 'Błąd, wyczyść płytę' 'Power set to 2' 'Power set to 5']


In [18]:
task.head(10)

,APPLIANCE_ID,MODEL_ID,COOKZONE_ID,MESSAGE_TEXT,EVENT_DATETIME
0,2867825,Bosch VF67,N/A,Deactivated,2023-07-05 18:12:45.119050
1,1419610,Bosch VF67,D,Ustaw czas!,2023-12-05 07:49:18.977865
2,5614226,Bosch VF67,B,Activated,2023-04-30 03:24:18.130563
3,5108603,Bosch VF67,D,Activated,2023-07-06 05:23:22.916729
4,4744854,Whirlpool XY12,N/A,Ustaw czas!,2023-05-17 22:59:43.737870
5,3341057,Whirlpool XY12,C,Activated,2023-11-08 20:10:35.002702
6,2719583,Bosch VF67,A,Power set to 8,2023-03-25 14:24:18.618088
7,2458591,Whirlpool XY12,B,"Błąd, wyczyść płytę",2023-12-19 17:14:44.215794
8,8078673,Bosch VF67,D,Power set to 2,2023-01-29 07:52:03.154776
9,1533224,Whirlpool XY12,C,Power set to 5,2023-07-25 08:04:14.923712


In [19]:
task.shape

(10, 5)

In [11]:
#QUESTION 1

In [28]:
task['EVENT_DATETIME'] = pd.to_datetime(task['EVENT_DATETIME'])
task_sorted = task.sort_values(by=['APPLIANCE_ID', 'EVENT_DATETIME'])

task_sorted['PREVIOUS_COOKZONE_ID'] = task_sorted.groupby('APPLIANCE_ID')['COOKZONE_ID'].shift(1)
changed_cookzones = task_sorted[
    (task_sorted['COOKZONE_ID'].notnull()) &
    (task_sorted['COOKZONE_ID'] != task_sorted['PREVIOUS_COOKZONE_ID'])
    ]
changed_cookzones = changed_cookzones[['APPLIANCE_ID', 'MODEL_ID', 'PREVIOUS_COOKZONE_ID', 'COOKZONE_ID', 'EVENT_DATETIME']]
changed_cookzones.reset_index(drop=True, inplace=True)
print(changed_cookzones)

   APPLIANCE_ID        MODEL_ID PREVIOUS_COOKZONE_ID COOKZONE_ID  \
0       1419610      Bosch VF67                  NaN           D   
1       1533224  Whirlpool XY12                  NaN           C   
2       2458591  Whirlpool XY12                  NaN           B   
3       2719583      Bosch VF67                  NaN           A   
4       2867825      Bosch VF67                  NaN         N/A   
5       3341057  Whirlpool XY12                  NaN           C   
6       4744854  Whirlpool XY12                  NaN         N/A   
7       5108603      Bosch VF67                  NaN           D   
8       5614226      Bosch VF67                  NaN           B   
9       8078673      Bosch VF67                  NaN           D   

              EVENT_DATETIME  
0 2023-12-05 07:49:18.977865  
1 2023-07-25 08:04:14.923712  
2 2023-12-19 17:14:44.215794  
3 2023-03-25 14:24:18.618088  
4 2023-07-05 18:12:45.119050  
5 2023-11-08 20:10:35.002702  
6 2023-05-17 22:59:43.737870  
7 2

In [33]:
#OBVIOUSLY THE RESULT IS NOT WHAT WE EXPECTED, BEACUSE DATA SHEET IS TOO SMALL, THERE ARE NO CHANGES IN COOKZONES FOR THE SAME APPLIANCE_ID, SO THE PREVIOUS_COOK_ID ALWAYS SHOWS NaN.

In [ ]:
#QUESTION 2

In [35]:
task['EVENT_DATETIME'] = pd.to_datetime(task['EVENT_DATETIME'])

meal = task[task['MESSAGE_TEXT'].isin(['Activated', 'Deactivated']) | (task['COOKZONE_ID'] != task['COOKZONE_ID'].shift())]

meal = meal.sort_values(by=['APPLIANCE_ID', 'EVENT_DATETIME'])

meal['MEAL_DURATION'] = meal.groupby('APPLIANCE_ID')['EVENT_DATETIME'].diff().fillna(pd.Timedelta(seconds=0))

meal_count = meal[(meal['MESSAGE_TEXT'] == 'Deactivated') & (meal['MEAL_DURATION'] <= pd.Timedelta(hours=1))].groupby('APPLIANCE_ID')['MESSAGE_TEXT'].count()

print("Estimated number of meals per appliance:")
print(meal_count)

Estimated number of meals per appliance:
APPLIANCE_ID
2867825    1
Name: MESSAGE_TEXT, dtype: int64


In [36]:
relevant_events = task[task['MESSAGE_TEXT'].isin(['Activated', 'Deactivated']) | (task['COOKZONE_ID'] != task['COOKZONE_ID'].shift())]
relevant_events = relevant_events.sort_values(by=['APPLIANCE_ID', 'EVENT_DATETIME'])
relevant_events['MEAL_DURATION'] = relevant_events.groupby('APPLIANCE_ID')['EVENT_DATETIME'].diff().fillna(pd.Timedelta(seconds=0))
meal_count = relevant_events[(relevant_events['MESSAGE_TEXT'] == 'Deactivated') & (relevant_events['MEAL_DURATION'] <= pd.Timedelta(hours=1))]\
    .groupby('APPLIANCE_ID')['MESSAGE_TEXT'].count()

print("Estimated number of meals per appliance:")
print(meal_count)

Estimated number of meals per appliance:
APPLIANCE_ID
2867825    1
Name: MESSAGE_TEXT, dtype: int64


In [ ]:
#The obtained result is of course correct, as we only got the value 1. This is because only for this ID there was a change in the status from "Activated" to "Deactivated".